# Integración con Google Trends
## Predicción de Clientes Fidelizables en E-commerce

### Objetivos de la Integración

Este notebook aborda la incorporación de datos externos de tendencias de búsqueda para enriquecer el modelo predictivo. Los objetivos específicos incluyen:

1. **Obtención de datos de Google Trends**
   - Selección de términos relevantes para e-commerce
   - Extracción de series temporales de interés
   - Manejo de limitaciones de la API

2. **Alineación temporal con transacciones**
   - Sincronización de períodos temporales
   - Agregación por períodos relevantes
   - Tratamiento de datos faltantes

3. **Creación de características derivadas**
   - Métricas estadísticas de tendencias por cliente
   - Variables de estacionalidad
   - Indicadores de momentum

### Justificación Teórica

La integración de datos de Google Trends se basa en la hipótesis de que las tendencias de búsqueda reflejan el interés del consumidor y pueden influir en el comportamiento de compra. Estudios previos han demostrado correlaciones significativas entre volúmenes de búsqueda y ventas en retail.

In [1]:
# Configuración inicial
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime, timedelta
import time
import warnings
warnings.filterwarnings('ignore')

# Importar pytrends para Google Trends
try:
    from pytrends.request import TrendReq
    pytrends_available = True
    print("PyTrends disponible para obtener datos reales")
except ImportError:
    pytrends_available = False
    print("PyTrends no disponible. Se utilizarán datos sintéticos")

print(f"Fecha de procesamiento: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

PyTrends disponible para obtener datos reales
Fecha de procesamiento: 2025-09-01 13:40:52


In [2]:
# Cargar datos de características de clientes
try:
    customer_data = pd.read_csv('../data/processed/customer_features.csv')
    print(f"Datos de clientes cargados: {len(customer_data):,} clientes")
except FileNotFoundError:
    print("Archivo de características no encontrado. Creando datos sintéticos...")
    # Crear datos sintéticos de clientes
    np.random.seed(42)
    n_customers = 1000
    
    customer_data = pd.DataFrame({
        'CustomerID': range(12346, 12346 + n_customers),
        'Recency': np.random.exponential(30, n_customers),
        'Frequency': np.random.poisson(5, n_customers) + 1,
        'Monetary': np.random.gamma(2, 100, n_customers),
        'Country': np.random.choice(['United Kingdom', 'France', 'Germany'], n_customers, p=[0.7, 0.15, 0.15])
    })

# Cargar datos de transacciones para alineación temporal
try:
    df_transactions = pd.read_csv('../data/processed/online_retail_clean.csv')
    df_transactions['InvoiceDate'] = pd.to_datetime(df_transactions['InvoiceDate'])
    print(f"Transacciones cargadas: {len(df_transactions):,} registros")
except FileNotFoundError:
    # Crear transacciones sintéticas
    np.random.seed(42)
    n_transactions = 5000
    
    df_transactions = pd.DataFrame({
        'CustomerID': np.random.choice(customer_data['CustomerID'], n_transactions),
        'InvoiceDate': pd.date_range('2010-12-01', '2011-12-09', periods=n_transactions),
        'Revenue': np.random.gamma(2, 50, n_transactions)
    })
    print(f"Transacciones sintéticas creadas: {len(df_transactions):,} registros")

# Definir período de análisis
start_date = df_transactions['InvoiceDate'].min().strftime('%Y-%m-%d')
end_date = df_transactions['InvoiceDate'].max().strftime('%Y-%m-%d')
print(f"Período de análisis: {start_date} a {end_date}")

Datos de clientes cargados: 4,338 clientes
Transacciones cargadas: 397,884 registros
Período de análisis: 2010-12-01 a 2011-12-09


## 1. Definición de Términos de Búsqueda

### Selección de Keywords

Los términos de búsqueda se seleccionan basándose en:
- Relevancia para el sector e-commerce
- Relación con comportamiento de compra
- Disponibilidad de datos históricos
- Variabilidad temporal suficiente

In [3]:
# Definir términos de búsqueda relevantes
search_terms = [
    'online shopping',
    'retail therapy',
    'gift shopping',
    'home decor',
    'christmas shopping'
]

print("TÉRMINOS DE BÚSQUEDA SELECCIONADOS")
print("=" * 40)
for i, term in enumerate(search_terms, 1):
    print(f"{i}. {term}")

print(f"\nJustificación de selección:")
print("- 'online shopping': Término general de e-commerce")
print("- 'retail therapy': Comportamiento de compra emocional")
print("- 'gift shopping': Compras estacionales y ocasionales")
print("- 'home decor': Categoría específica del dataset")
print("- 'christmas shopping': Estacionalidad navideña")

# Configuración para Google Trends
geo_location = 'GB'  # Reino Unido (principal mercado del dataset)
timeframe = f'{start_date} {end_date}'

print(f"\nConfiguración de búsqueda:")
print(f"Ubicación geográfica: {geo_location}")
print(f"Marco temporal: {timeframe}")

TÉRMINOS DE BÚSQUEDA SELECCIONADOS
1. online shopping
2. retail therapy
3. gift shopping
4. home decor
5. christmas shopping

Justificación de selección:
- 'online shopping': Término general de e-commerce
- 'retail therapy': Comportamiento de compra emocional
- 'gift shopping': Compras estacionales y ocasionales
- 'home decor': Categoría específica del dataset
- 'christmas shopping': Estacionalidad navideña

Configuración de búsqueda:
Ubicación geográfica: GB
Marco temporal: 2010-12-01 2011-12-09


## 2. Obtención de Datos de Google Trends

### Metodología de Extracción

La extracción de datos sigue un protocolo robusto que incluye:
- Manejo de errores y reintentos
- Pausas para evitar límites de API
- Validación de datos obtenidos
- Fallback a datos sintéticos si es necesario

In [4]:
def get_google_trends_data(keywords, timeframe, geo='GB', retries=3):
    """
    Obtiene datos de Google Trends con manejo robusto de errores
    
    Parameters:
    keywords (list): Lista de términos de búsqueda
    timeframe (str): Período de tiempo
    geo (str): Código de país
    retries (int): Número de reintentos
    
    Returns:
    pd.DataFrame: Datos de tendencias
    """
    if not pytrends_available:
        return None
    
    pytrends = TrendReq(hl='en-US', tz=360)
    trends_data = pd.DataFrame()
    
    for keyword in keywords:
        success = False
        attempts = 0
        
        while not success and attempts < retries:
            try:
                pytrends.build_payload([keyword], cat=0, timeframe=timeframe, geo=geo, gprop='')
                interest_over_time = pytrends.interest_over_time()
                
                if not interest_over_time.empty:
                    interest_over_time = interest_over_time.drop('isPartial', axis=1, errors='ignore')
                    interest_over_time.columns = [f'trends_{keyword.replace(" ", "_")}']
                    
                    if trends_data.empty:
                        trends_data = interest_over_time
                    else:
                        trends_data = trends_data.join(interest_over_time, how='outer')
                
                success = True
                print(f"Datos obtenidos para '{keyword}'")
                
            except Exception as e:
                attempts += 1
                print(f"Error obteniendo '{keyword}' (intento {attempts}): {str(e)[:50]}...")
                if attempts < retries:
                    time.sleep(5)
            
            time.sleep(2)  # Pausa entre requests
    
    return trends_data

# Intentar obtener datos reales de Google Trends
print("OBTENCIÓN DE DATOS DE GOOGLE TRENDS")
print("=" * 40)

trends_data = get_google_trends_data(search_terms, timeframe, geo_location)

if trends_data is not None and not trends_data.empty:
    print(f"Datos reales obtenidos: {trends_data.shape[0]} períodos")
    print(f"Columnas: {list(trends_data.columns)}")
else:
    print("No se pudieron obtener datos reales. Creando datos sintéticos...")
    
    # Crear datos sintéticos con patrones realistas
    date_range = pd.date_range(start=start_date, end=end_date, freq='W')
    np.random.seed(42)
    
    trends_data = pd.DataFrame({'date': date_range})
    
    for term in search_terms:
        col_name = f'trends_{term.replace(" ", "_")}'
        
        # Crear tendencia base con estacionalidad
        base_trend = 50 + 30 * np.sin(2 * np.pi * np.arange(len(date_range)) / 52)
        
        # Añadir ruido y variabilidad
        noise = np.random.normal(0, 10, len(date_range))
        
        # Añadir picos estacionales para términos específicos
        if 'christmas' in term:
            # Pico en diciembre
            christmas_boost = np.where(
                trends_data['date'].dt.month == 12, 40, 0
            )
            base_trend += christmas_boost
        
        trends_data[col_name] = np.clip(base_trend + noise, 0, 100).astype(int)
    
    trends_data.set_index('date', inplace=True)
    print(f"Datos sintéticos creados: {trends_data.shape[0]} períodos")
    print(f"Columnas: {list(trends_data.columns)}")

# Mostrar primeras filas
print(f"\nPrimeras 5 filas de datos de tendencias:")
display(trends_data.head())

OBTENCIÓN DE DATOS DE GOOGLE TRENDS
Datos obtenidos para 'online shopping'
Datos obtenidos para 'retail therapy'
Datos obtenidos para 'gift shopping'
Datos obtenidos para 'home decor'
Datos obtenidos para 'christmas shopping'
Datos reales obtenidos: 54 períodos
Columnas: ['trends_online_shopping', 'trends_retail_therapy', 'trends_gift_shopping', 'trends_home_decor', 'trends_christmas_shopping']

Primeras 5 filas de datos de tendencias:


,trends_online_shopping,trends_retail_therapy,trends_gift_shopping,trends_home_decor,trends_christmas_shopping
date,,,,,
2010-11-28,100,0,68,58,50
2010-12-05,98,0,100,35,58
2010-12-12,88,0,100,42,66
2010-12-19,76,0,90,37,100
2010-12-26,73,0,51,55,33
